In [3]:
from datasets import load_dataset
expanded_plan_dataset = load_dataset("vdaita/commitpackft-small-patches-expanded", split="train")

In [1]:
from superdocs_python.utils.prompts import EDITBLOCK_PROMPTS, AIDER_UDIFF_PLAN_AND_EXECUTE_PROMPT

generation_inst = AIDER_UDIFF_PLAN_AND_EXECUTE_PROMPT
generation_inst_sr = EDITBLOCK_PROMPTS

In [4]:
from superdocs_python.utils.diff_utils import parse_diff

def reformat_diff(diff_string):
    sr_blocks = parse_diff(diff_string)
    generated_changes = ""
    for block in sr_blocks:
        generated_changes += f"""```
{block.filepath}
<<<<<<< ORIGINAL
{block.search_block}
=======
{block.replace_block}
>>>>>>> UPDATED
```\n"""
    return generated_changes

In [5]:
from datasets import concatenate_datasets
import difflib
import re

languages = ["python", "jsx", "kotlin", "vue", "xml", "javascript", "java", "c#", "ruby", "c", "c++", "typescript", "clojure", "dart"]
generated_datasets = []

desired_columns = ["input", "input_inst", "output"]

for language in languages:
    dataset = load_dataset("bigcode/commitpackft", language, split="train")
    dataset = dataset.shuffle(seed=42).select(range(60, 90)) # Used the same seed as the other shuffler that created the plans
    def process_row(row):
        udiff = "\n".join(difflib.unified_diff(row['old_contents'].splitlines(), row['new_contents'].splitlines(), fromfile=row['old_file'], tofile=row['new_file'], n=3))
        udiff = f"```diff\n{udiff}\n```"
        udiff = re.sub("@@.*@@", "@@...@@", udiff)
        
    
        query = f"[{row['old_file']}]\n```\n{row['old_contents']}```\nGenerate a unified diff patch that completes the following task: \n{row['subject']}"
        
        return {"input": query, "input_inst": query + "\n" + generation_inst, "output": reformat_diff(udiff)}
    dataset = dataset.map(lambda x: process_row(x))
    dataset = dataset.select_columns(desired_columns)
    generated_datasets.append(dataset)

def process_row_no_code(row):
    query = f"[{row['old_file']}]\n```\n{row['old_contents']}```\nGenerate a unified diff patch that completes the following task: \n{row['one_sentence_instruction']}\nFollow this plan:\n{row['plan_no_code']}"
    return {"input": query, "input_inst": query + "\n" + generation_inst, "output": reformat_diff(row["patch"])}

def process_row_code(row):
    query = f"[{row['old_file']}]\n```\n{row['old_contents']}```\nGenerate a unified diff patch that completes the following task: \n{row['one_sentence_instruction']}\nFollow this plan:\n{row['plan_with_code']}"
    return {"input": query, "input_inst": query + "\n" + generation_inst, "output": reformat_diff(row["patch"])}

no_code_ds = expanded_plan_dataset.map(lambda x: process_row_no_code(x))
with_code_ds = expanded_plan_dataset.map(lambda x: process_row_code(x))
no_code_ds = no_code_ds.select_columns(desired_columns)
with_code_ds = with_code_ds.select_columns(desired_columns)

generated_datasets.append(no_code_ds)
generated_datasets.append(with_code_ds)
dataset = concatenate_datasets(generated_datasets)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]


NameError: name 'generation_inst' is not defined

In [11]:
dataset.push_to_hub("vdaita/commitpackft-patches")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.64it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/vdaita/commitpackft-patches/commit/27119a1f457d7e6963560841f9eb2628a025b5e6', commit_message='Upload dataset', commit_description='', oid='27119a1f457d7e6963560841f9eb2628a025b5e6', pr_url=None, pr_revision=None, pr_num=None)